In [1]:
##Libraries needed
import numpy as np
import pandas as pd
import h5py 
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

In [2]:
#Mask to get the SWGO array
#Function to get the WCD coordinates
def BuildArrayMask_Index(Rmax1=160,Rmax2=565,FF1=0.8,FF2=0.04):
    #Input variables:
    #Rmax1: Minimum radius of the outer array
    #Rmax2: Maximum radius of the array
    #FF1: Fill factor of the inner array
    #FF2: Fill factor of the outer array
    #Create array with the empty mask
    pixels_x = pixels_y = 283    
    #Grid with r of each value in the row
    grid = np.arange(-Rmax2,Rmax2,4)
    grid=grid+1
    r_mask = np.ones((pixels_x**2)).reshape((pixels_x,pixels_y))
    for i in range(pixels_x):
        for j in range(pixels_y):
            r_mask[i,j] = np.sqrt(grid[i]**2 + grid[j]**2)
    #Define spacing using the fill factor
    #Inner array
    spacing_FF1 = np.round(np.sqrt((8*np.pi)/(FF1*np.sqrt(3))),2)
    dx_FF1 = int(round((spacing_FF1/4),0))
    dy_FF1 = int(round((spacing_FF1*0.5*np.sqrt(3)/4),0))
    #Outer array
    spacing_FF2 = np.round(np.sqrt((8*np.pi)/(FF2*np.sqrt(3))),2)
    dx_FF2 = int(round((spacing_FF2/4),0))
    dy_FF2 = int(round((spacing_FF2*0.5*np.sqrt(3)/4),0))
    #Build mask for Inner array
    rows = np.arange(0,pixels_x,dy_FF1)
    #Loop over rows
    counter = 0
    mask_inner = np.zeros((pixels_x**2)).reshape((pixels_x,pixels_y))
    for j in rows:
        odd_dis = int((counter%2)*np.ceil(dx_FF1*0.5))
        mask_inner[j,odd_dis::dx_FF1+1] = 1
        #If FF>=50% keep odd rows full of stations to increase FF
        if FF1>0.5: 
            if (counter%2)!=0: mask_inner[j,:] = 1
        counter+=1
    #Change values out of the array for 0 
    mask_inner[r_mask>Rmax1]=0
    #Build mask for outer array
    rows = np.arange(0,pixels_x,dy_FF2)
    #Loop over rows
    counter = 0
    mask_outer = np.zeros((pixels_x**2)).reshape((pixels_x,pixels_y))
    for j in rows:
        odd_dis = int((counter%2)*np.ceil(dx_FF2*0.5))
        mask_outer[j,odd_dis::dx_FF2+1] = 1
        counter+=1
    #Change values out of the array for 0 
    mask_outer[(r_mask<=Rmax1)|(r_mask>Rmax2)]=0
    mask = mask_inner+mask_outer
    return mask



In [3]:

from sklearn.preprocessing import MinMaxScaler

def LogMinMaxScaler(data):
    #Loop over events
    for i in range(len(data)):
        #Apply log scale to values different to 0
        data[i][data[i]!=0] = np.log10(data[i][data[i]!=0])
        #Scale from 1 to 10 values different to 0
        scaler = MinMaxScaler(feature_range=(1,10))
        scaler.fit(data[i][data[i]!=0].reshape(-1,1))
        data[i][data[i]!=0] = scaler.transform(data[i][data[i]!=0].reshape(-1,1)).reshape(-1)
        del scaler
    return data

In [4]:

primary = "gamma"
size = 560 #detector radius, area: 1km^2

if (primary=="gamma"):
    filename_train = "Datosphoton_alt5200m_qgsii_fluka_r560m_3PMTs_40-60TeV_N725.h5"
    particle = "Gamma"
elif (primary=="proton"):
    filename_train = "Datosproton_alt5200m_qgsii_fluka_r560m_3PMTs_40-60TeV-GammaERange_N1955.h5"
    particle = "Proton"


In [5]:
#Read data
f = h5py.File(filename_train, mode = "r")

group = f["data"]

#Read shower parameters
InfoDF = pd.read_hdf(filename_train, key = "info")
ID_showers = InfoDF.iloc[:,0].values
E0_train = InfoDF.iloc[:,1].values
theta0_train = InfoDF.iloc[:,2].values
Nmuons_train = InfoDF.iloc[:,3].values

#Read input for the algorithm
#X_train: channel1 -> e.m. energy, channel2 -> particles, channel3 -> muons
X_train = group[()]
Y_train = InfoDF.iloc[:,-1].values


#Close file
f.close()

In [6]:

#Apply mask
mask_to_apply = BuildArrayMask_Index()   


X_train = mask_to_apply*X_train



#New normalization data
X_train = LogMinMaxScaler(X_train)



In [7]:
#Saving images

for i in range(len(X_train)):
    try:
        fig = plt.figure(figsize=(4,4))
        plt.imshow(X_train[i,:,:], alpha=0.8, cmap='Reds',norm=mpl.colors.LogNorm())
        plt.axis('off')
        filename = './fillfactor/'+str(primary)+'/'+str(primary)+'_id'+str(i)+'.jpg'
        plt.savefig(filename)
        plt.close(fig)
    except: 
        print('Error in i = '+str(i))
        pass